In [1]:
# NOTE: export SPARK_DRIVER_MEMORY=1g

In [1]:
import keras
from keras.applications import ResNet50
from keras.applications import imagenet_utils

Using TensorFlow backend.


* For resnet, we only support NCHW and Keras 1.2.2 for now

In [2]:
keras.backend.set_image_dim_ordering("th")

* Create ResNet50 using Keras

In [3]:
keras_model = ResNet50(weights="imagenet")

/usr/local/lib/python2.7/dist-packages/keras/applications/resnet50.py:231: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


* Init Analytics-zoo and spark environment

In [4]:
from bigdl.keras.backend import *
from bigdl.transform.vision.image import *
from zoo.common.nncontext import *
sc = get_nncontext()


Using /home/lizhichao/bin/spark-2.1.0-bin-hadoop2.7
Adding /home/lizhichao/bin/god/zoo/dist/lib/zoo-0.1.0-SNAPSHOT-jar-with-dependencies.jar to BIGDL_JARS
Adding /home/lizhichao/bin/god/zoo/dist/lib/zoo-0.1.0-SNAPSHOT-jar-with-dependencies.jar to SPARK_CLASSPATH


* Load a Keras definition and weights.

In [5]:
bmodel = DefinitionLoader.from_kmodel(keras_model)
WeightLoader.load_weights_from_kmodel(bmodel, keras_model)

creating: createInput


/home/lizhichao/bin/god/zoo/dist/lib/zoo-0.1.0-SNAPSHOT-python-api.zip/bigdl/keras/converter.py:791: UserWarning: Cannot find dim_ordering from json definition. Using the default instead.


creating: createSequential
creating: createPadding
creating: createPadding
creating: createPadding
creating: createPadding
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createSpatialBatchNormalization
creating: createCAddTable
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
c

* Load data into ImageFrame

In [6]:
image_frame = ImageFrame.read("image-net", sc=sc)

* Define prepocess transformer

In [7]:
transformer = Pipeline([BytesToMat(), Resize(256, 256),
                        CenterCrop(224, 224),
                        ChannelNormalize(123.68, 103.939, 116.779),
                        MatToTensor(), ImageFrameToSample()])

creating: createBytesToMat
creating: createResize
creating: createCenterCrop
creating: createChannelNormalize
creating: createMatToTensor
creating: createImageFrameToSample
creating: createPipeline


* Run prediction

In [8]:
preds_frame = bmodel.predict_image(image_frame.transform(transformer))

* Show result of the first image

In [9]:
preds = preds_frame.get_predict().take(1)

for pre in preds:
    path, pre_score = pre
    P = imagenet_utils.decode_predictions(np.expand_dims(pre_score, axis=0))
    print("Prediction for {}".format(path))
    # Display the top-5 prediction
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("    {}. {}: {:.2f}%".format(i + 1, label, prob * 100))

Prediction for file:/home/lizhichao/bin/god/tzoo/notebooks/part2/image-net/158516268_796c3e047d.jpg
    1. sorrel: 96.77%
    2. ox: 0.72%
    3. llama: 0.52%
    4. gazelle: 0.42%
    5. basenji: 0.23%


* While the origin image is
![scalars](image-net/158516268_796c3e047d.jpg)